In [1]:
filePath = 'audio/seven.wav'

- separate

In [ ]:
from spleeter.separator import Separator

def separate(path: str):
    print('start separate')

    separateMode = 5
    # setting spleeter separator
    separator = Separator(f'spleeter:{separateMode}stems')

    # start separate
    separator.separate_to_file(path, f'result_{separateMode}stems')

    print('end separate')

In [ ]:
separate(filePath)

In [ ]:
file = filePath[:filePath.find('.')]

In [ ]:
import librosa

def getInfo(path: str) -> (int, int):

    audio_data, sr = librosa.load(path)

    tempo = int(librosa.beat.tempo(y=audio_data, sr=sr))

    return (tempo, sr)


In [ ]:
BPM, sr = getInfo(filePath)

In [ ]:
import librosa
import numpy as np

def getFreq(path: str, BPM: int, sr: int):
    output_path = path.replace('.wav', '_Freq.txt')
    output = open(output_path, 'w')

    audio_data, sr = librosa.load(path, sr=sr)

    slice_rate = 2584 / (BPM * 4)
    # get the magnitudes
    pitches, magnitudes = librosa.piptrack(y=audio_data, sr=sr)
    # turn the amplitude to db
    magnitudes_db = librosa.amplitude_to_db(magnitudes)
    # get the max db's frequency
    frequencies = np.argmax(magnitudes_db, axis=0)

    pitch_times = librosa.times_like(frequencies, sr=sr)

    # store every time's freq
    pitch_list = []
    for time, pitch in zip(pitch_times, frequencies):
        pitch_list.append((time, pitch))

    cnt = 1
    while True:
        index = int(slice_rate * cnt)
        if index > len(pitch_list): break
        output.write(f"{pitch_list[index][1]}\n")
        cnt += 1

    output.close()

In [ ]:
def getSheet(path: str):
    FreqTable = 'FreqTable.txt'
    file = open(FreqTable, 'r')
    data = file.readlines()
    table = []
    for i in data:
        name, freq = i.split(' ')
        table.append((name, float(freq)))
    file.close()

    audio = open(path, 'r')
    outputPath = path.replace('Freq', 'sheet')
    out = open(outputPath, 'w')
    data = audio.readlines()
    for i in data:
        i = float(i)
        if i == 0: out.write('_ ')
        else:
            for j in range(len(table)-1):
                if table[j][1] <= i < table[j+1][1]:
                    out.write(f'{table[j][0]} ')
                    break
    audio.close()
    out.close()

In [ ]:
kinds = ['piano', 'vocals', 'drums', 'bass', 'other']
for kind in kinds:
    splitPath = f'result_5stems/{file}/{kind}.wav'
    getFreq(splitPath, BPM, sr) # put the file in result/file/kind_Freq.txt
    FreqPath = f'result_5stems/{file}/{kind}_Freq.txt'
    getSheet(FreqPath)